In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import numpy as np

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [2]:
# preparamos los datos
dataloader = {
    'train': torch.utils.data.DataLoader(torchvision.datasets.MNIST('../data', train=True, download=True,
                       transform=torchvision.transforms.Compose([
                            torchvision.transforms.ToTensor(),
                            torchvision.transforms.Normalize((0.1307,), (0.3081,))
                            ])
                      ), batch_size=2048, shuffle=True, pin_memory=True),
    'test': torch.utils.data.DataLoader(torchvision.datasets.MNIST('../data', train=False,
                   transform=torchvision.transforms.Compose([
                        torchvision.transforms.ToTensor(),
                        torchvision.transforms.Normalize((0.1307,), (0.3081,))
                        ])
                     ), batch_size=2048, shuffle=False, pin_memory=True)
}

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw




Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw

Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [3]:
# definimos el modelo

def block(c_in, c_out, k=3, p=1, s=1, pk=2, ps=2):
    return torch.nn.Sequential(
        torch.nn.Conv2d(c_in, c_out, k, padding=p, stride=s),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(pk, stride=ps)
    )

class CNN(torch.nn.Module):
  def __init__(self, n_channels=1, n_outputs=10):
    super().__init__()
    self.conv1 = block(n_channels, 64)
    self.conv2 = block(64, 128)
    self.fc = torch.nn.Linear(128*7*7, n_outputs)

  def forward(self, x):
    x = self.conv1(x)
    x = self.conv2(x)
    x = x.view(x.shape[0], -1)
    x = self.fc(x)
    return x

In [4]:
# entrenamos el modelo

def fit(model, dataloader, epochs=5):
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    criterion = torch.nn.CrossEntropyLoss()
    for epoch in range(1, epochs+1):
        model.train()
        train_loss, train_acc = [], []
        bar = tqdm(dataloader['train'])
        for batch in bar:
            X, y = batch
            X, y = X.to(device), y.to(device)
            optimizer.zero_grad()
            y_hat = model(X)
            loss = criterion(y_hat, y)
            loss.backward()
            optimizer.step()
            train_loss.append(loss.item())
            acc = (y == torch.argmax(y_hat, axis=1)).sum().item() / len(y)
            train_acc.append(acc)
            bar.set_description(f"loss {np.mean(train_loss):.5f} acc {np.mean(train_acc):.5f}")
        bar = tqdm(dataloader['test'])
        val_loss, val_acc = [], []
        model.eval()
        with torch.no_grad():
            for batch in bar:
                X, y = batch
                X, y = X.to(device), y.to(device)
                y_hat = model(X)
                loss = criterion(y_hat, y)
                val_loss.append(loss.item())
                acc = (y == torch.argmax(y_hat, axis=1)).sum().item() / len(y)
                val_acc.append(acc)
                bar.set_description(f"val_loss {np.mean(val_loss):.5f} val_acc {np.mean(val_acc):.5f}")
        print(f"Epoch {epoch}/{epochs} loss {np.mean(train_loss):.5f} val_loss {np.mean(val_loss):.5f} acc {np.mean(train_acc):.5f} val_acc {np.mean(val_acc):.5f}")

In [5]:
model = CNN()
fit(model, dataloader)

  0%|          | 0/30 [00:00<?, ?it/s]

Epoch 1/5 loss 0.60715 val_loss 0.18959 acc 0.81944 val_acc 0.94623


  0%|          | 0/30 [00:00<?, ?it/s]

Epoch 2/5 loss 0.13743 val_loss 0.08256 acc 0.95921 val_acc 0.97462


  0%|          | 0/30 [00:00<?, ?it/s]

Epoch 3/5 loss 0.07887 val_loss 0.05794 acc 0.97673 val_acc 0.98276


  0%|          | 0/30 [00:00<?, ?it/s]

Epoch 4/5 loss 0.06058 val_loss 0.04704 acc 0.98221 val_acc 0.98516


val_loss 0.04610 val_acc 0.98413: 100%|██████████| 5/5 [00:02<00:00,  2.14it/s]

Epoch 5/5 loss 0.05083 val_loss 0.04610 acc 0.98465 val_acc 0.98413


In [6]:
#guardar modelo 1 forma recomendable para hacer chekpint ya que hay que instanciar un modelo.

PATH = './checkpoint.pt'

torch.save(model.state_dict(),PATH)

In [ ]:
""from google.colab import drive
drive.mount('/content/drive')""

In [ ]:
##carga MODELO forma 1

model.load_state_dict(torch.load(PATH))
model.eval()

In [8]:
def evaluate(model, dataloader):
    model.eval()
    model.to(device)
    bar = tqdm(dataloader['test'])
    acc = []
    with torch.no_grad():
        for batch in bar:
            X, y = batch
            X, y = X.to(device), y.to(device)
            y_hat = model(X)
            acc.append((y == torch.argmax(y_hat, axis=1)).sum().item() / len(y))
            bar.set_description(f"acc {np.mean(acc):.5f}")

In [9]:
evaluate(model,dataloader)

acc 0.98413: 100%|██████████| 5/5 [00:02<00:00,  2.03it/s]


In [10]:
#segunda forma de guardar
torch.save(model,'model.pt')


In [11]:
#cargar modelo 2

model= torch.load('model.pt')
model.eval()



CNN(
  (conv1): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Linear(in_features=6272, out_features=10, bias=True)
)

In [12]:
#exportar modelos para que no dependan de python llamdo tracing no es flexible ya que si tengo condicionales en capas solo agarra 1

x=torch.rand(32,1,28,28)
traced_model = torch.jit.trace(model.cpu(), x)
traced_model.save('model.zip')

In [13]:
#caragado de modelo 1 forma
loaded_model= torch.jit.load('model.zip')
evaluate(loaded_model,dataloader)

acc 0.98413: 100%|██████████| 5/5 [00:02<00:00,  2.00it/s]


In [14]:
##2forma de exportar que es el scripting

scripted_model = torch.jit.script(model.cpu())
scripted_model.save('model.zip')




In [15]:
#cargado de segunda forma
loaded_model = torch.jit.load('model.zip')
evaluate(loaded_model,dataloader)

acc 0.98413: 100%|██████████| 5/5 [00:02<00:00,  2.03it/s]


In [28]:
#esta manera es como els cripting pero ams general el scripting en para entornos pytorch y torchscript
#este se llama ONNX
x = torch.rand(32, 1, 28, 28)
y = model.cpu()(x)

# exportamos el modelo
torch.onnx.export(model,                     # el modelo
                  x,                         # un ejemplo del input
                  "model.onnx",              # el nombre del archivo para guardar el modelo
                  export_params=True,        # guardar los pesos de la red
                  opset_version=10,          # versión de ONNX
                  do_constant_folding=True,  # optimizaciones
                  input_names = ['input'],   # nombre de los inputs
                  output_names = ['output'], # nombre de los outputs
                  dynamic_axes={'input' : {0 : 'batch_size'},    # ejes con longitud variable (para poder usar diferentes tamaños de batch)
                                'output' : {0 : 'batch_size'}})



In [32]:
!pip install onnxruntime
import onnxruntime

     |████████████████████████████████| 4.1MB 11.4MB/s 


In [33]:


def onnx_evaluate(model, dataloader):
    # cargarmos el modelo
    ort_session = onnxruntime.InferenceSession(model)
    bar = tqdm(dataloader['test'])
    acc = []
    with torch.no_grad():
        for batch in bar:
            X, y = batch
            X, y = X.numpy(), y.numpy()
            # generamos los inputs
            ort_inputs = {ort_session.get_inputs()[0].name: X}
            # extraemos los outputs
            ort_outs = ort_session.run(None, ort_inputs)[0]
            acc.append((y == np.argmax(ort_outs, axis=1)).mean())
            bar.set_description(f"acc {np.mean(acc):.5f}")

In [34]:
onnx_evaluate("model.onnx", dataloader)

acc 0.98413: 100%|██████████| 5/5 [00:07<00:00,  1.48s/it]


In [ ]:
#ejmeplos de pre y pos procesado se 
class Preprocessing(torch.nn.Module):
  def __init__(self):
    super().__init__()
  def forward
  #esperamos un batch de imagenes sin normalizar
  #normalizamos
  x=(x/255.)
  x= (x-0.1307)/ 0.3081 

  #dimensiones -> [Bs,canal,h,w]
  x=x.unsqeeze(1)
  # en imagenes en color, harias un permute idk
  return x


class postprocessing(torch.nn.Module):
  def __init__(self):
    super().__init__()
    self.softmax = torch.nn.Softmax(dim=1)
  def forward(self, x):
    #devolvemos distribucion de propabilidad
    #y clase con mayos porbabilidad
    return self.softmax(x), torch.argmax(x,dim=1)






